In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
import time
from classes.dataset import Dataset
from libs.location import *

2023-09-27 07:22:24.057237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 07:22:24.575071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.metrics import accuracy_score,classification_report

In [18]:
X_FILE_NAME = sys.argv[1]
Y_FILE_NAME = sys.argv[2]

In [4]:
X = np.load(f'{DATASET_LOCATION_PATH}/{X_FILE_NAME}')
y = np.load(f'{DATASET_LOCATION_PATH}/{Y_FILE_NAME}')

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
ds = Dataset()

##Initialising Dataset


In [7]:
chexnet_model = ds.create_chexnet_model(with_weights=False)
chexnet_model.load_weights(f'{MODELS_CHECKPOINT_LOCATION_PATH}/xray/model_chexnet_2023-09-26.E09-VL0.47-A0.76.h5')

##Creating Custom ChexNet Model


2023-09-27 07:22:30.541016: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 07:22:30.589153: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 07:22:30.589329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
chexnet_model.compile(
    optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [9]:
score = chexnet_model.evaluate(X_test, y_test, verbose = 1)
score

2023-09-27 07:22:35.651456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-27 07:22:35.805962: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-27 07:22:36.056571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


167/167 [==============================] - 15s 75ms/step - loss: 0.4726 - accuracy: 0.7774


[0.47255995869636536, 0.7774030566215515]

In [10]:
y_pred = chexnet_model.predict(X_test)

167/167 [==============================] - 13s 72ms/step


In [14]:
y_pred.shape

(5337, 2)

In [15]:
y_train_final = [np.argmax(i) for i in y_train]
y_test_final = [np.argmax(i) for i in y_test]
y_pred_final = [np.argmax(i) for i in y_pred]

In [17]:
print(classification_report(y_pred_final,y_test_final))

              precision    recall  f1-score   support

           0       0.94      0.80      0.87      4798
           1       0.24      0.54      0.33       539

    accuracy                           0.78      5337
   macro avg       0.59      0.67      0.60      5337
weighted avg       0.87      0.78      0.81      5337

